In [1]:
!pip install pymysql

In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
try:
    from lightgbm import LGBMRegressor
except ModuleNotFoundError:
    import os
    os.system('pip install lightgbm')
    from lightgbm import LGBMRegressor
try:
    from catboost import CatBoostRegressor
except ModuleNotFoundError:
    import os
    os.system('pip install catboost')
    from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.exceptions import NotFittedError
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [5]:
!pip install xgboost

In [30]:
# MySQL connection details
# username= 'root'
# password= 'root%401234'
# host = 'localhost'
# port = '3306'
# database = 'dev_days'

# # Create SQLAlchemy engine
# engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [7]:
def sales_vs_stock():
    df_sales = pd.read_csv('film&tv-sale.csv');
    df_stockin = pd.read_csv('film&tv-stockin.csv');
    
    # Current date
    current_date = datetime.now()
    
    # Start date of the same month one year ago
    year_ago = (current_date - relativedelta(years=1)).replace(day=1)
    
    # Last date of the previous month in the current year
    month_ago = (current_date.replace(day=1) - relativedelta(days=1))

    # print(year_ago, month_ago)
    
    df_sales['OrderDt'] = pd.to_datetime(df_sales['OrderDt'])
    df_sales_year_ago = df_sales[(df_sales['OrderDt'] >= year_ago) & (df_sales['OrderDt'] <= month_ago)].copy()
    df_sales_year_ago['month'] = df_sales['OrderDt'].dt.strftime('%B-%Y')
    
    df_stockin['OrderDt'] = pd.to_datetime(df_stockin['OrderDt'])
    df_stockin_year_ago = df_stockin[(df_stockin['OrderDt'] >= year_ago) & (df_stockin['OrderDt'] <= month_ago)].copy()
    df_stockin_year_ago['month'] = df_stockin['OrderDt'].dt.strftime('%B-%Y')
    
    sales_result = df_sales_year_ago.groupby('month', as_index=False).agg({'Qty': 'sum'})
    stockin_result = df_stockin_year_ago.groupby('month', as_index=False).agg({'Qty': 'sum'})
    
    sales_vs_stock = pd.DataFrame({
        'MonthYear': pd.to_datetime(sales_result['month'], format='%B-%Y'),
        'Sales': sales_result['Qty'],
        'Stock': stockin_result['Qty'],
    }).sort_values(by='MonthYear');

    store_data('sales_vs_stock', sales_vs_stock)

In [9]:
def box_stock():
    raw_df = pd.read_csv('dev_day_data_11Nov.csv', sep="\t");
    df = pd.DataFrame({
        'StoreId': raw_df['BranchID'],
        'ProductID': raw_df['Boxid'],
        'QuantityOnHand': raw_df['QuantityOnHand']
    })
    
    # Write DataFrame to MySQL table
    store_data('box_stock', df)

In [11]:
def map_box_to_category(df):
    box_categories = pd.DataFrame({
        'ProductID': df['ProductId'],
        'Category': df['box_category']
    }).drop_duplicates()
    
    store_data('box_categories', box_categories)

In [17]:
df = pd.read_csv('film&tv-sale.csv').drop_duplicates()

def feature_engineering(df):
    df['OrderDt'] = pd.to_datetime(df['OrderDt'])
    df['year'] = df['OrderDt'].dt.year
    df['month'] = df['OrderDt'].dt.month
    df['week'] = df['OrderDt'].dt.isocalendar().week
    df['day'] = df['OrderDt'].dt.day
    df['day_of_week'] = df['OrderDt'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_black_friday'] = ((df['month'] == 11) & (df['day'] >= 24) & (df['day'] <= 30)).astype(int)
    df['is_cyber_monday'] = ((df['month'] == 11) & (df['day'] == 27)).astype(int)
    df['is_christmas_week'] = ((df['month'] == 12) & (df['day'] >= 21) | (df['month'] == 1) & (df['day'] <= 7)).astype(int)
    
    for lag in [7, 14, 30, 60, 90, 180]:
        df[f'lag_{lag}_qty'] = df['Qty'].shift(lag)
        df[f'rolling_{lag}_mean_qty'] = df['Qty'].rolling(window=lag, min_periods=1).mean()
        df[f'ema_{lag}_qty'] = df['Qty'].ewm(span=lag, adjust=False).mean()
    
    df['sales_growth_7d'] = (df['Qty'] - df['lag_7_qty']) / df['lag_7_qty']
    df['sales_growth_30d'] = (df['Qty'] - df['lag_30_qty']) / df['lag_30_qty']
    df.fillna(0, inplace=True)
    return df

In [3]:
# def category_sales_stock():

SyntaxError: incomplete input (2424897636.py, line 1)

In [21]:
# def get_data(query):

#     # Fetch data into a Pandas DataFrame
#     df = pd.read_sql_query(query, engine)

#     return df;
import os
print(os.listdir())  # Check if 'film&tv-sale.csv' exists


['.DS_Store', 'predicition-model-Copy1.ipynb', 'film&tv-sale.csv', 'predicition-model.ipynb', '.ipynb_checkpoints', 'film&tv-sale_og.csv', 'film&tv-stockin.csv']


In [ ]:
# def store_data(table_name, df):
#     # Write DataFrame to MySQL table
#     df.to_sql(table_name, con=engine, if_exists='replace', index=False)

#     print(f"Data saved in table successfully in {table_name}..!!!")

In [19]:
df = feature_engineering(df)

# Encode categorical features
label_encoders = {}
categorical_cols = ['storeid', 'ProductId', 'box_category']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Expand training data
df = df[df['OrderDt'] >= '2021-01-01']

# Train-test split
test_start_date = '2023-11-24'
test_end_date = '2023-11-30'
test_data = df[(df['OrderDt'] >= test_start_date) & (df['OrderDt'] <= test_end_date)].copy()
test_data['OrderDt'] = pd.to_datetime(test_data['OrderDt']) + pd.DateOffset(years=1)
train_data = df.copy()

features = [
    'storeid', 'ProductId', 'year', 'month', 'week', 'day', 'day_of_week', 'is_weekend',
    'is_black_friday', 'is_cyber_monday', 'is_christmas_week',
    'lag_7_qty', 'lag_14_qty', 'lag_30_qty', 'lag_60_qty', 'lag_90_qty', 'lag_180_qty',
    'rolling_7_mean_qty', 'rolling_14_mean_qty', 'rolling_30_mean_qty', 'rolling_60_mean_qty', 'rolling_90_mean_qty', 'rolling_180_mean_qty',
    'ema_7_qty', 'ema_14_qty', 'ema_30_qty', 'ema_60_qty', 'ema_90_qty', 'ema_180_qty',
    'sales_growth_7d', 'sales_growth_30d', 'box_category']

X_train = train_data[features]
y_train = train_data['Qty']
X_test = test_data[features]
y_test = test_data['Qty']

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Evaluate Model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest MSE: {mse_rf}, R² Score: {r2_rf}")


Random Forest MSE: 1.922209363905619e-09, R² Score: 0.9999997632971049
